In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
from torch.utils.data import DataLoader
from torch.nn.functional import softmax
from tqdm import tqdm

In [4]:
model_name = "finiteautomata/bertweet-base-sentiment-analysis"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


In [5]:
model.eval()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(130, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [42]:
from data.SentimentAnalysisDataset import get_sentiment_dataset, create_collate_fn
train = get_sentiment_dataset("train", tokenizer)
test = get_sentiment_dataset("test", tokenizer)

collate_fn = create_collate_fn(tokenizer)

train_loader = DataLoader(train, batch_size=16, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test, batch_size=16, shuffle=False, collate_fn=collate_fn)

In [44]:
def predict_dataset(model, loader):
    predictions = []
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    with torch.no_grad():
        for batch in tqdm(loader):
            batch = {key: val.to(device) for key, val in batch.items()}
            outputs = model(**batch)
            probs = softmax(outputs.logits, dim=1)
            preds = torch.argmax(probs, dim=1).cpu().numpy()
            predictions.extend(preds)

    return predictions

# Get predictions
preds = predict_dataset(model, test_loader)

100%|██████████| 747/747 [07:52<00:00,  1.58it/s]


In [47]:
mapping={0:"negative", 1:"neutral", 2:"positive"}
preds_mapped = [mapping[p] for p in preds]

In [48]:
preds_mapped

['neutral',
 'neutral',
 'negative',
 'negative',
 'negative',
 'positive',
 'negative',
 'neutral',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'negative',
 'neutral',
 'negative',
 'positive',
 'neutral',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'positive',
 'positive',
 'negative',
 'negative',
 'neutral',
 'negative',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'positive',
 'neutral',
 'negative',
 'neutral',
 'positive',
 'positive',
 'positive',
 'neutral',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'negative',
 'neutral',
 'negative',
 'neutral',
 'neutral',
 'negative',
 'positive',
 'neutral',
 'negative',
 'neutral',
 'neutral',
 'positive',
 'neutral',
 'neutral',
 'positive',
 'neutral',
 'positive',
 'negative',
 'negative',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'positive',
 'neutral',
 'negative',
 'neu

In [55]:
from submissions.SubmissionCreation import create_submission
create_submission(preds_mapped)